# **Geonet Data Extract**
The purpose of the following code is used to extract raw ambient virbation data from collected by GNS sensors and create text files that can be read in MPIT (Operational Modal Analysis Toolbox). Outputs of Acceleration vs Time Plots and Fourier Transforms can also be generated to aid in identifying good sets of data to expedite the process.

Note: The function "analysis_select" relates to data collected at the BNZ Building Site.

Raw ambient vibration data collected from GNS can be found here:
ftp://ftp.geonet.org.nz/strong/raw/building/

**Current Code Version: 10.1.0**

## **Import Relevant Python Libraries**

In [ ]:
import pandas as pd
import os
!pip install eqsig
import eqsig
import matplotlib.pyplot as plt
import numpy as np

##**Functions**

###Determine Analysis Type and Global Direction
This function processes the users choices in the analysis type (Whole Building vs One Pier [Pier 3]) and also the global direction for acceleration response that they would like to analyse. The outputted channels are the associated channels and acceleration directions in the channels that will be required to conduct their chosen analysis. The sensors were orientated with Azimuths of 37&deg; and 307&deg; which meant the X and Y directions of some of the sensors were pointing in different directions. Shown in the possible Channel/Direction combinations below is the associated combinations for the global X (Across the Piers) and Y (Along the Piers) directions that were set.

In [ ]:
#Select channels to process depending on intended global direction and analysis type
def dir_analysis_select(analysis_type, global_direction):
        if analysis_type == 1 and global_direction == "X":
            channels = ['4 X','5 X','6 X','7 Y','8 X','9 Y','10 Y','11 X','12 Y','13 X','14 Y','15 Y']
        elif analysis_type == 1 and global_direction == "Y":
            channels = ['4 Y','5 Y','6 Y','7 X','8 Y','9 X','10 X','11 Y','12 X','13 Y','14 X','15 X']
        elif analysis_type == 1 and global_direction == "Z":
            channels = ['4 Z','5 Z','6 Z','7 Z','8 Z','9 Z','10 Z','11 Z','12 Z','13 Z','14 Z','15 Z']
        elif analysis_type == 2 and global_direction == "X":
            channels = ['4 X','5 X','6 X','7 Y','8 X','9 Y','10 Y','11 X','15 Y']
        elif analysis_type == 2 and global_direction == "Y":
            channels = ['4 Y','5 Y','6 Y','7 X','8 Y','9 X','10 X','11 Y','15 X']
        else:
            channels = ['4 Z','5 Z','6 Z','7 Z','8 Z','9 Z','10 Z','11 Z','15 Z']
        
        return channels
    


###Detrend acceleration data
In the data sets there maybe systematic errors or deviations from a zero mean. This can effect the modes retrieved from signal processing and so this function finds the average acceleration value across the entire response and adjusts every data point to ensure that the average acceleration value in the response is zero. 

In [ ]:
#Takes a list of acceleration data and removes a constant offset trend.
def detrend(raw_acc = []):
    mean = round(sum(raw_acc)/len(raw_acc),6)
    for z in range(len(raw_acc)):
            raw_acc[z] = f'{raw_acc[z] - mean:.6f}'
    return raw_acc
       


###Generate a DataFrame
This data frame resembles a table, which has rows of acceleration response for each increment of time (0.005sec) for a 200Hz Sample Rate, and columns representing X,Y and Z acceleration responses for each sensor. 

In [ ]:
#Generate a data frame containing the accleration data.
def generate_df(file_name):
      
    #Open raw data files
    raw_data = open(file_name, 'r', encoding = 'latin1')
    lines = raw_data.readlines()
    raw_data.close()

    #Store the file name:
    file_name = file_name[:-4]
    #Store relevant information into variables to aid in extracting the data
    num_lines = len(lines)
    label_length = 28
    info_rows = 37
    sample_rate = int((lines[20])[label_length:])
    
    #Channel Data
    no_channels = (lines[21])[-21:]
    no_channels = int(no_channels[0:2])

    #Create time data for data frame
    counter = 37
    time_list = []
    time = 0
	
    while counter < num_lines:
        time_list.append("{:.3f}".format(time))
        counter += 1
        time += 1/int(sample_rate)
    
    timedf = {"Time":time_list[0:num_lines - info_rows]}

    #Create the data frame with the time in the first column
    df= pd.DataFrame(timedf)

    #Establish variables due to character spacing in text file
    j = 28
    k = 37
    l = 1

    for channel_number in range(1,no_channels + 1):
        counter = 0
        value_list_x = []
        value_list_y = []
        value_list_z = []
		
		#Collect accelerations in each direction and store to value_list_x,y,z
        while (counter < num_lines - info_rows):
            value_list_x.append(round(float(lines[counter + info_rows][j : k]),6))
            value_list_y.append(round(float(lines[counter + info_rows][j + 11 : k + 11]),6))
            value_list_z.append(round(float(lines[counter + info_rows][j + 22: k + 22]),6))
            counter += 1
            
        #Detrend Data using the established Detrending Function
        value_list_x = detrend(value_list_x)
        value_list_y = detrend(value_list_y)
        value_list_z = detrend(value_list_z)
        
        #Append value_list_x,y,z to DataFrame  
        df['Channel ' + str(channel_number) + ' X'] = value_list_x[0 : num_lines - info_rows]
        df['Channel ' + str(channel_number) + ' Y'] = value_list_y[0 : num_lines - info_rows]
        df['Channel ' + str(channel_number) + ' Z'] = value_list_z[0 : num_lines - info_rows]
        
        l += 3
        j += 39
        k += 39
        
    #Reverse X direction acceleration recordings for sensors orientated at 37 azimuth to align measurements  
    azimuth37sensors = ["Channel 7 X", "Channel 9 X", "Channel 10 X", "Channel 12 X", "Channel 14 X", "Channel 15 X"]
    
    for sensor in azimuth37sensors:
        df[sensor] = df[sensor].astype(float)
        df[sensor] *= -1
        pd.options.display.float_format = '{:.6f}'.format
        
    return df

##Output Data Frame Information into useable TXT files
This function outputs the data frame information into .txt files that can be used in MPIT to conduct system ID. The outputted text file will have a column for time, and then subsequent appended columns showing the relevant sensor data for the chosen analysis type (Whole Building or Pier 3) and the global direction. 

In [ ]:
#Convert raw GeoNet Data into tab delimited .txt files for the depending on the chosen global X,Y or Z direction.    
def data_to_CSV(df, channel_list):
    #Write the acceleration values into text files:
    header = ["Time"]  
    for j in channel_list:
        header.append("Channel " + j)
        df.to_csv(file_name + " Acceleration " + global_direction + ".txt", columns = header, float_format = '%.6f', index = False, sep = "\t")




###Generate Acceleration vs Time and Fourier Transform Plots
This function draws Acceleration vs Time and Fourier Transform plots for Channel 9 X direction acceleration response. This channel was chosen since it is on Level 5 of the BNZ building and so would experience large movements compared to channels lower in the building. These plots can help identify the days, that have acceleration response data that shows the modes accurately.

In [ ]:
#Draws the Acc vs. Time + FDD
def draw(df, file_name):
        #Convert Data Frame to Float
        df[0:] = df[0:].astype(float)
                    
        #Create an Signal object
        acc = df['Channel 9 X'].to_numpy()
        dt = df.iloc[1,0]
        asig = eqsig.AccSignal(acc, dt, label='name_of_record')
        
        #Produce Fourier Spectrum
        plt.plot(asig.fa_frequencies, abs(asig.fa_spectrum), c='b')
        asig.smooth_fa_frequencies = np.logspace(-1, 1, 50)
        plt.title('Fourier Spectrum ' + file_name[:-4])
        plt.xlabel('Frequency [Hz]')
        plt.ylabel('Fourier Amplitude [m/s]')
        plt.savefig('/content/drive/Shared drives/Civil 705A B P4P Project/Data Processing/FTP Data/Ferry Terminal Data/Fourier/' + file_name + '_FFD.png', dpi=300, bbox_inches='tight')
        
        #Draw Acceleration vs. Time Plot
        df[0:] = df[0:].astype(float)
        df.plot(x='Time', y='Channel 9 X', color='r', linewidth=0.25)
        plt.title('Time vs. Acceleration ' + file_name[:-4])
        plt.xlabel('Time (s)')
        plt.ylabel('Acceleration (ms.-2)')
        plt.savefig('/content/drive/Shared drives/Civil 705A B P4P Project/Data Processing/FTP Data/Ferry Terminal Data/AVT/' + file_name + '.png', dpi=300, bbox_inches='tight')
        
        #Close plots
        plt.close('all')
        


##**Main Code**
Ensure that the variable 'dir' is updated to be the file path of the location where the raw data to be analysed is. The folder name could be of the form '20XX CMFS'

For each .cmf file:
   1. Generate a data frame containing all selected channels.
   2. Split accelertions into X,Y,Z direction txt files.
   3. Draw the FDD and Acc. vs Time

In [ ]:
#Ask for Analysis Type
analysis_type = int(input("[1] = Whole Building Analysis,\n[2] = Single Bay Analysis, \nEnter Analysis Type: "))
global_direction = input("[X] = Across the three piers\n[Y] = Along the piers towards the harbour\n[Z] = Vertical\nEnter Global Direction: ")
print("Processing Data...")

'''
IMPORTANT
Update the 'dir' file path below to the location of the CMF's. Ensure that the folder name is NINE characters long i.e. 20XX CMFS
'''

dir = str("/content/drive/Shared drives/Civil 705A B P4P Project/Data Processing/FTP Data/Ferry Terminal Data/2015/2015 CMFS")
#Creates files required to save plots into
if os.path.exists(dir + '/AVT') == 'False':
    avt_path = os.path.join(dir[:-10],'AVT')
    os.mkdir(avt_path)
if os.path.exists(dir + '/Fourier') == 'False':
    fourier_path = os.path.join(dir[:-10],'Fourier')
    os.mkdir(fourier_path)

for file in os.listdir(dir):
    if file.endswith(".cmf"):
        channels = dir_analysis_select(analysis_type, global_direction)
        file_name = os.path.join(dir, file)
        df = generate_df(file_name)    
        data_to_CSV(df, channels)
        draw(df, file) 
        print(file + " completed...")

print("All files completed")
print("IMPORTANT: Remember to copy AVT and Fourier plots to appropriate folder locations, otherwise they will be overwritten in the next run")

[1] = Whole Building Analysis,
[2] = Single Bay Analysis, 
Enter Analysis Type: 1
[X] = Across the three piers
[Y] = Along the piers towards the harbour
[Z] = Vertical
Enter Global Direction: X
Processing Data...
All files completed
Remember to copy AVT and Fourier plots to appropriate folder locations, otherwise they will be overwritten in the next run


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
